# Read CTDdat.ncdf 

this R notebook reads (and cleans up) the CTD ncdf file that is provided on  
  
https://www.bco-dmo.org/dataset/3092

This .ncdf file is a collection of monthly CTD runs at CARIACO Ocean Time Series Station.

In [1]:
require(dplyr, warn.conflicts = FALSE);
#require(ggplot2);
require(cowplot, warn.conflicts = FALSE); # for nicer plots, includes ggplot2
require(chron, warn.conflicts = FALSE);
require(ncdf4, warn.conflicts = FALSE);

Loading required package: dplyr
Loading required package: cowplot
Loading required package: ggplot2
Loading required package: chron
Loading required package: ncdf4


In [2]:
nctdname <- "DATA_raw/CTD.netcdf"
# open a NetCDF file
nctdin <- nc_open(nctdname)

print(nctdin)

CruiseID <- ncvar_get(nctdin,"Cruise_ID")
date <- ncvar_get(nctdin,"Date")
month <- ncvar_get(nctdin,"Month")
press <- ncvar_get(nctdin,"press")
depth <- ncvar_get(nctdin,"depth")
sigma_t <- ncvar_get(nctdin,"sigma_t")
sigma_0 <- ncvar_get(nctdin, "sigma_0")
potemp <- ncvar_get(nctdin, "potemp")
sal <- ncvar_get(nctdin, "sal")
temp <- ncvar_get(nctdin, "temp")
lat <- ncvar_get(nctdin, "lat")
lon <- ncvar_get(nctdin, "lon")
beam_att <- ncvar_get(nctdin, "beam_att")
beam_cp <- ncvar_get(nctdin, "beam_cp")
fluor_chla <- ncvar_get(nctdin, "fluor_chla")

nc_close(nctdin)

File DATA_raw/CTD.netcdf (NC_FORMAT_CLASSIC):

     19 variables (excluding dimension variables):
        char Cruise_ID[maxlen_,unlimited]   
        double lon[unlimited]   
        double lat[unlimited]   
        double Date[unlimited]   
        double Year[unlimited]   
        double Month[unlimited]   
        double press[unlimited]   
        double depth[unlimited]   
        double temp[unlimited]   
        double sal[unlimited]   
        double potemp[unlimited]   
        double sigma_t[unlimited]   
        double sigma_0[unlimited]   
        double O2_ml_L[unlimited]   
        double O2_umol_kg[unlimited]   
        char beam_cp[maxlen_,unlimited]   
        char beam_att[maxlen_,unlimited]   
        char fluor_CTD[maxlen_,unlimited]   
        char fluor_chla[maxlen_,unlimited]   

     2 dimensions:
        unlimited  Size:250474   *** is unlimited ***
        maxlen_  Size:513


In [3]:
# date format
DateString <- date
Date <- as.POSIXlt(as.character(date), format="%Y%m%d")

days <- Date$yday

# reformat NAs
press[press<0] <- NA
sigma_t[sigma_t<0] <- NA
sigma_0[sigma_0<0] <- NA
potemp[potemp<0] <- NA
sal[sal<0] <- NA
temp[temp<0] <- NA
beam_att[beam_att<0] <- NA
beam_cp[beam_cp<0] <- NA
fluor_chla[fluor_chla<0] <- NA

press[press>1e30] <- NA
sigma_t[sigma_t>1e30] <- NA
sigma_0[sigma_0>1e30] <- NA
potemp[potemp>1e30] <- NA
sal[sal>1e30] <- NA
temp[temp>1e30] <- NA
beam_att[beam_att>1e30] <- NA
beam_cp[beam_cp>1e30] <- NA
fluor_chla[fluor_chla>1e30] <- NA

# create data.frame (without NAs)
CTDdat <- data.frame(CruiseID,Date,month,days,press,depth,lat,lon,sigma_t,sigma_0,potemp,sal,temp,beam_att,beam_cp,fluor_chla)
head(CTDdat)

CruiseID,Date,month,days,press,depth,lat,lon,sigma_t,sigma_0,potemp,sal,temp,beam_att,beam_cp,fluor_chla
93HG_001,1995-11-08,11,311,2.011,2,10.5,-64.67,23.764,23.764,27.522,36.612,27.522,NA,NA,NA
93HG_001,1995-11-08,11,311,4.023,4,10.5,-64.67,23.775,23.775,27.488,36.612,27.489,NA,NA,NA
93HG_001,1995-11-08,11,311,6.034,6,10.5,-64.67,23.782,23.782,27.464,36.614,27.465,NA,NA,NA
93HG_001,1995-11-08,11,311,8.046,8,10.5,-64.67,23.786,23.787,27.456,36.613,27.458,NA,NA,NA
93HG_001,1995-11-08,11,311,10.057,10,10.5,-64.67,23.787,23.787,27.452,36.613,27.454,NA,NA,NA
93HG_001,1995-11-08,11,311,12.069,12,10.5,-64.67,23.787,23.788,27.449,36.612,27.452,NA,NA,NA


In [4]:
tail(CTDdat)

,CruiseID,Date,month,days,press,depth,lat,lon,sigma_t,sigma_0,potemp,sal,temp,beam_att,beam_cp,fluor_chla
250469,93HG_199,2013-03-13,3,71,1318,1306.393,10.5,-64.67,26.411,26.466,17.061,36.259,17.288,0.030,0.394,NA
250470,93HG_199,2013-03-13,3,71,1319,1307.381,10.5,-64.67,26.411,26.466,17.061,36.259,17.288,0.031,0.395,NA
250471,93HG_199,2013-03-13,3,71,1320,1308.369,10.5,-64.67,26.411,26.466,17.060,36.258,17.289,0.030,0.394,NA
250472,93HG_199,2013-03-13,3,71,1321,1309.357,10.5,-64.67,26.411,26.466,17.060,36.259,17.289,0.030,0.394,NA
250473,93HG_199,2013-03-13,3,71,1322,1310.345,10.5,-64.67,26.411,26.466,17.060,36.258,17.289,0.032,0.396,NA
250474,93HG_199,2013-03-13,3,71,1323,1311.333,10.5,-64.67,26.411,26.466,17.060,36.258,17.289,0.030,0.394,NA


In [5]:
# Write CSV in R
write.csv(CTDdat, file = "DATA_processed/CTDdat_processed.csv")